In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from itertools import combinations

# Plots

In [ ]:
nayearly = pd.read_table('/home/users/clelland/Compare and contrast/NAyearly.csv', sep=",", index_col=0).T
sibyearly = pd.read_table('/home/users/clelland/Compare and contrast/Sibyearly.csv', sep=",", index_col=0).T
sousibyearly = pd.read_table('/home/users/clelland/Compare and contrast/SouSibyearly.csv', sep=",", index_col=0).T

In [ ]:
nayearlyred = nayearly.drop(['Fire_cci masked50', 'AVHRR-LTDR mask50'], axis=1)
nayearlyred = nayearlyred.rename(columns={'Fire_cci masked80': 'FireCCI51', 'AVHRR-LTDR mask30': 'FireCCILT11', 'MOD14A1': 'MODIS Active Fires'})
sibyearlyred = sibyearly.drop(['Fire_cci masked50', 'AVHRR-LTDR mask50'], axis=1)
sibyearlyred = sibyearlyred.rename(columns={'Fire_cci masked80': 'FireCCI51', 'IKI':'SRBA', 'Talucci':'Talucci et al.', 'AVHRR-LTDR mask30': 'FireCCILT11', 'MOD14A1': 'MODIS Active Fires'})
sousibyearlyred = sousibyearly.drop(['Fire_cci masked50', 'AVHRR-LTDR mask50'], axis=1)
sousibyearlyred = sousibyearlyred.rename(columns={'Fire_cci masked80': 'FireCCI51', 'AVHRR-LTDR mask30': 'FireCCILT11', 'MOD14A1': 'MODIS Active Fires'})

In [ ]:
pd.set_option('display.precision', 2)
col_set = ['#8c564b', '#bcbd22', '#17becf', '#d62728', '#ff7f0e', '#9467bd', '#2ca02c', '#1f77b4', '#7f7f7f', '#e377c2']

## NA

In [ ]:
# Data
headers = list(nayearlyred.columns)
values = nayearlyred.loc['SUM'].values.tolist()

# Calculate percentage differences
data = {}
for i in range(len(values)):
    for j in range(len(values)):
        diff = (values[i] - values[j]) / values[j] * 100
        data[(headers[i], headers[j])] = diff

# Convert data to DataFrame
df = pd.DataFrame(data.values())
reshaped_df = df.values.reshape(10, 10)
new_df = pd.DataFrame(reshaped_df)
headers = ['FireCCI51', 'MCD64A1', 'GABAM', 'MTBS/NBAC', 'GFED4s', 'GFED5', 'ABoVE-FED', 'FireCCILT11', 'MODIS Active Fires', 'ABBA']
new_df.set_index(pd.Index(headers), inplace=True)
new_df = new_df.set_axis(headers, axis=1)
new_df

# Mask diagonal values for plot
new_df_ma = np.ma.masked_where(new_df == 0, new_df)
#cmap = plt.get_cmap('coolwarm').set_bad(color="yellow") # Run this line first time then re-run with it hashed out

# Plot
plt.figure(figsize=(10, 8))
#plt.imshow(new_df_ma, cmap=cmap, interpolation='nearest', vmin=-100, vmax=100) # to do with the masking - can be hashed out on re-run
plt.imshow(new_df_ma, cmap='coolwarm', interpolation='nearest', vmin=-100, vmax=100)

# Add value annotations
for i in range(len(headers)):
    for j in range(len(headers)):
        if i == j:
            plt.text(i, j, 'X', va='center', ha='center', color='black', fontsize=8)
        else:
            plt.text(j, i, f'{new_df.values[i, j]:.2f}%', ha='center', va='center', color='black', fontsize=8)

plt.xticks(np.arange(len(headers)), headers, rotation=90)
plt.yticks(np.arange(len(headers)), headers)
plt.title('North America % Difference of 2002-2016 Total Burned Area')
plt.colorbar(label='% Difference')
plt.tight_layout()
#plt.savefig('napercdiff0216.tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
nayearlytable = nayearlyred.copy()

# Caculate average across of all datasets across each year
nayearlytable['AVG'] = nayearlytable.mean(numeric_only=True, axis=1)

# Calculate mean for the years 2002-2016
mean_row = nayearlytable.iloc[1:16].mean().to_frame().T
mean_row.index = ['Average']
nayearlytable = pd.concat([nayearlytable, mean_row])

# Calculate standard deviation of each column for the years 2002-2016 using rows 2-16
std_dev_row = nayearlytable.iloc[1:16].std().to_frame().T
std_dev_row.index = ['SD']
nayearlytable = pd.concat([nayearlytable, std_dev_row])

# Calculate the standard deviation for each row and create a new column
nayearlytable['STD'] = nayearlytable.iloc[:, :-1].std(axis=1)

nayearlytable

In [ ]:
nayearlyredred = nayearlyred.copy()
nayearlyredred.drop('AVG', axis=1, inplace=True)
nayearlyredred = nayearlyredred.iloc[1:16]

# Initialize a list to store results
results = []

# Iterate through each pair of columns
for col1, col2 in combinations(nayearlyredred.columns, 2):
    # Perform T-test
    t_stat, p_value = stats.ttest_ind(nayearlyredred[col1], nayearlyredred[col2], equal_var=False)
    # Format p_value to 4 decimal places
    p_value = round(p_value, 3)
    # Store the result
    results.append((col1, col2, t_stat, p_value))

# Convert results to a DataFrame
na_pval = pd.DataFrame(results, columns=["Column1", "Column2", "T-Statistic", "P-Value"])

# Print the results
print(na_pval)

# Print only the statistically relevant results
filtered_nap = na_pval[na_pval['P-Value'] < 0.05]
print(filtered_nap)

In [ ]:
boxplot = nayearlyred.iloc[1:16].boxplot(column=['GABAM', 'FireCCI51', 'FireCCILT11', 'MCD64A1', 'MODIS Active Fires', 'GFED5', 'MTBS/NBAC', 'GFED4s', 'ABBA', 'ABoVE-FED'], grid=False, sym='x', vert=False, showmeans=True)
plt.xlabel('Burned area (Mha)')
#plt.savefig('FigS5 (NA).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
nayearlyred['AVG'].iloc[0] = np.nan
nayearlyred['AVG'].iloc[16:] = np.nan

In [ ]:
f = plt.figure(figsize=[20,6])
ax = f.add_subplot(111)
nayearlyredgraph = nayearlyred.copy()
nayearlyredgraph = nayearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'MTBS/NBAC', 'ABoVE-FED']]
nayearlyredgraph = nayearlyredgraph.iloc[:20]
nayearlyred = nayearlyred.rename(columns={'AVG': str()})
nayearlyred[''].plot(kind='line', ax=ax, color='black', marker='o')
nayearlyredgraph.plot(kind='bar', width=0.8, ax=ax)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(nayearlyredgraph.columns))
ax.set_xlabel('year')
ax.set_ylabel('Burned area (Mha)')
#plt.savefig('Fig5 (Sib bar).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
f = plt.figure(figsize=[18,6])
ax = f.add_subplot(111)
nayearlyredgraph = nayearlyred.copy()
nayearlyredgraph['Average'] = nayearlyredgraph.mean(axis=1)
nayearlyredgraph['Average'].iloc[0] = np.nan
nayearlyredgraph['Average'].iloc[16:] = np.nan
nayearlyredgraph = nayearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'MTBS/NBAC', 'ABoVE-FED', 'Average']]
nayearlyredgraph = nayearlyredgraph.iloc[:20]
(nayearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'MTBS/NBAC', 'ABoVE-FED']]
 .plot(kind='line', ax=ax, lw=2, color=col_set))
nayearlyredgraph['Average'].plot(kind='line', ax=ax, color='black', marker='o', lw=4, ls='--', ms=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(nayearlyredgraph.columns))
ax.set_xticks(np.arange(len(nayearlyredgraph.index)))
ax.set_xticklabels(nayearlyredgraph.index)
ax.set_xlabel('year')
ax.set_ylabel('Burned area (Mha)')
#plt.savefig('Fig2 (NA line).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
nayearlybar = nayearlyred.copy()
nayearlybar = nayearlybar.round(2)

f = plt.figure(figsize=[8,6])
ax = f.add_subplot(111)
#nayearlybar = nayearlybar[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'MTBS/NBAC', 'ABoVE-FED']]
nayearlybar = nayearlybar[['GABAM', 'FireCCI51', 'FireCCILT11', 'MCD64A1', 'MODIS Active Fires', 'GFED5', 'MTBS/NBAC', 'GFED4s', 'ABBA', 'ABoVE-FED']]
col_na = ['#17becf', '#8c564b', '#9467bd', '#bcbd22', '#2ca02c', '#ff7f0e', '#7f7f7f', '#d62728', '#1f77b4', '#e377c2']
#nayearlybar.loc['SUM'].plot(kind='bar', ax=ax, color=col_sum)
nayearlybar.loc['SUM'].plot(kind='bar', ax=ax, color=col_na)
ax.set_ylabel('Burned area (Mha)')
ax.bar_label(ax.containers[0])
#plt.savefig('FigS4 (NA).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

## Sib

In [ ]:
# Data
headers = list(sibyearlyred.columns)
values = sibyearlyred.loc['SUM'].values.tolist()

# Calculate percentage differences
data = {}
for i in range(len(values)):
    for j in range(len(values)):
        diff = (values[i] - values[j]) / values[j] * 100
        data[(headers[i], headers[j])] = diff

# Convert data to DataFrame
df = pd.DataFrame(data.values())
reshaped_df = df.values.reshape(10, 10)
new_df = pd.DataFrame(reshaped_df)
headers = ['FireCCI51', 'MCD64A1', 'GABAM', 'Talucci et al.', 'SRBA', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA']
new_df.set_index(pd.Index(headers), inplace=True)
new_df = new_df.set_axis(headers, axis=1)
new_df

# Mask diagonal values for plot
new_df_ma = np.ma.masked_where(new_df == 0, new_df)
#cmap = plt.get_cmap('coolwarm').set_bad(color="yellow") # Run this line first time then re-run with it hashed out

# Plot
plt.figure(figsize=(10, 8))
#plt.imshow(new_df_ma, cmap=cmap, interpolation='nearest', vmin=-100, vmax=100) # to do with the masking - can be hashed out on re-run
plt.imshow(new_df_ma, cmap='coolwarm', interpolation='nearest', vmin=-100, vmax=100)

# Add value annotations
for i in range(len(headers)):
    for j in range(len(headers)):
        if i == j:
            plt.text(i, j, 'X', va='center', ha='center', color='black', fontsize=8)
        else:
            plt.text(j, i, f'{new_df.values[i, j]:.2f}%', ha='center', va='center', color='black', fontsize=8)

plt.xticks(np.arange(len(headers)), headers, rotation=90)
plt.yticks(np.arange(len(headers)), headers)
plt.title('Central/Northern Siberia % Difference of 2006-2016 Total Burned Area')
plt.colorbar(label='% Difference')
plt.tight_layout()
#plt.savefig('sibpercdiff0616.tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
sibyearlytable = sibyearlyred.copy()

# Caculate average across of all datasets across each year
sibyearlytable['AVG'] = sibyearlytable.mean(numeric_only=True, axis=1)

# Calculate mean for the years 2006-2016
mean_row = sibyearlytable.iloc[5:16].mean().to_frame().T
mean_row.index = ['Average']
sibyearlytable = pd.concat([sibyearlytable, mean_row])

# Calculate standard deviation of each column for the years 2006-2016 using rows 6-16
std_dev = sibyearlytable.iloc[5:16].std().to_frame().T
std_dev.index = ['Standard Deviation']
sibyearlytable = pd.concat([sibyearlytable, std_dev])

# Calculate the standard deviation for each row and create a new column
sibyearlytable['STD'] = sibyearlytable.iloc[:, :-1].std(axis=1)

sibyearlytable

In [ ]:
sibyearlyredred = sibyearlyred.copy()
sibyearlyredred.drop('AVG', axis=1, inplace=True)
sibyearlyredred = sibyearlyredred.iloc[5:16]

# Initialize a list to store results
results = []

# Iterate through each pair of columns
for col1, col2 in combinations(sibyearlyredred.columns, 2):
    # Perform T-test
    t_stat, p_value = stats.ttest_ind(sibyearlyredred[col1], sibyearlyredred[col2], equal_var=False)
    # Format p_value to 4 decimal places
    p_value = round(p_value, 3)
    # Store the result
    results.append((col1, col2, t_stat, p_value))

# Convert results to a DataFrame
sib_pval = pd.DataFrame(results, columns=["Column1", "Column2", "T-Statistic", "P-Value"])

# Print the results
print(sib_pval)

# Print only the statistically relevant results
filtered_nap = sib_pval[sib_pval['P-Value'] < 0.05]
print(filtered_nap)

In [ ]:
boxplot = sibyearlyred.iloc[5:16].boxplot(column=['GABAM', 'SRBA', 'FireCCILT11', 'GFED4s', 'MCD64A1', 'FireCCI51', 'ABBA', 'Talucci et al.', 'GFED5', 'MODIS Active Fires'], grid=False, sym='x', vert=False, showmeans=True)
plt.axhline(y=6.5, color='r', linestyle='--')
plt.axhline(y=7.5, color='r', linestyle='--')
plt.xlabel('Burned area (Mha)')
#plt.savefig('FigS7 (Sib).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
sibyearlyred['AVG'].iloc[0:5] = np.nan
sibyearlyred['AVG'].iloc[16:] = np.nan

In [ ]:
f = plt.figure(figsize=[20,6])
ax = f.add_subplot(111)
sibyearlyredgraph = sibyearlyred.copy()
sibyearlyredgraph = sibyearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'Talucci et al.', 'SRBA']]
sibyearlyredgraph = sibyearlyredgraph.iloc[:20]
sibyearlyred = sibyearlyred.rename(columns={'AVG': str()})
sibyearlyred[''].plot(kind='line', ax=ax, color='black', marker='o')
sibyearlyredgraph.plot(kind='bar', width=0.8, ax=ax)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(nayearlyredgraph.columns))
ax.set_xlabel('year')
ax.set_ylabel('Burned area (Mha)')
#plt.savefig('Fig5 (Sib bar).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
f = plt.figure(figsize=[18,6])
ax = f.add_subplot(111)
sibyearlyredgraph = sibyearlyred.copy()
sibyearlyredgraph['Average'] = sibyearlyredgraph.mean(axis=1)
sibyearlyredgraph['Average'].iloc[0:5] = np.nan
sibyearlyredgraph['Average'].iloc[16:] = np.nan
sibyearlyredgraph = sibyearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'Talucci et al.', 'SRBA', 'Average']]
sibyearlyredgraph = sibyearlyredgraph.iloc[:20]
(sibyearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'Talucci et al.', 'SRBA']]
 .plot(kind='line', ax=ax, lw=2, color=col_set))
sibyearlyredgraph['Average'].plot(kind='line', ax=ax, color='black', marker='o', lw=4, ls='--', ms=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(sibyearlyredgraph.columns))
ax.set_xticks(np.arange(len(sibyearlyredgraph.index)))
ax.set_xticklabels(sibyearlyredgraph.index)
ax.set_xlabel('year')
ax.set_ylabel('Burned area (Mha)')
#plt.savefig('Fig5 (Sib line).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
sibyearlybar = sibyearlyred.copy()
sibyearlybar = sibyearlybar.round(2)

f = plt.figure(figsize=[8,6])
ax = f.add_subplot(111)
#sibyearlybar = sibyearlybar[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'ABBA', 'Talucci et al.', 'SRBA']]
sibyearlybar = sibyearlybar[['GABAM', 'SRBA', 'FireCCILT11', 'GFED4s', 'MCD64A1', 'FireCCI51', 'ABBA', 'Talucci et al.', 'GFED5', 'MODIS Active Fires']]
col_sib = ['#17becf', '#e377c2', '#9467bd', '#d62728', '#bcbd22', '#8c564b', '#1f77b4', '#7f7f7f', '#ff7f0e', '#2ca02c']
#sibyearlybar.loc['SUM'].plot(kind='bar', ax=ax, color=col_set)
sibyearlybar.loc['SUM'].plot(kind='bar', ax=ax, color=col_sib)
ax.set_ylabel('Burned area (Mha)')
ax.bar_label(ax.containers[0])
#plt.savefig('FigS6 (Sib).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

## SouSib

In [ ]:
# Data
headers = list(sousibyearlyred.columns)
values = sousibyearlyred.loc['SUM'].values.tolist()

# Calculate percentage differences
data = {}
for i in range(len(values)):
    for j in range(len(values)):
        diff = (values[i] - values[j]) / values[j] * 100
        data[(headers[i], headers[j])] = diff

# Convert data to DataFrame
df = pd.DataFrame(data.values())
reshaped_df = df.values.reshape(7, 7)
new_df = pd.DataFrame(reshaped_df)
headers = ['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires']
new_df.set_index(pd.Index(headers), inplace=True)
new_df = new_df.set_axis(headers, axis=1)
new_df

# Mask diagonal values for plot
new_df_ma = np.ma.masked_where(new_df == 0, new_df)
cmap = plt.get_cmap('coolwarm').set_bad(color="yellow") # Run this line first time then re-run with it hashed out

# Plot
plt.figure(figsize=(8, 6))
#plt.imshow(new_df_ma, cmap=cmap, interpolation='nearest', vmin=-100, vmax=100) # to do with the masking - can be hashed out on re-run
plt.imshow(new_df_ma, cmap='coolwarm', interpolation='nearest', vmin=-100, vmax=100)

# Add value annotations
for i in range(len(headers)):
    for j in range(len(headers)):
        if i == j:
            plt.text(i, j, 'X', va='center', ha='center', color='black', fontsize=8)
        else:
            plt.text(j, i, f'{new_df.values[i, j]:.2f}%', ha='center', va='center', color='black', fontsize=8)
            
plt.xticks(np.arange(len(headers)), headers, rotation=90)
plt.yticks(np.arange(len(headers)), headers)
plt.title('Southern Siberia % Difference of 2001-2016 Total Burned Area')
plt.colorbar(label='% Difference')
plt.tight_layout()
#plt.savefig('sousibpercdiff0116.tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
sousibyearlytable = sousibyearlyred.copy()

# Caculate average across of all datasets across each year
sousibyearlytable['AVG'] = sousibyearlytable.mean(numeric_only=True, axis=1)

# Calculate mean for the years 2001-2016
mean_row = sousibyearlytable.iloc[:16].mean().to_frame().T
mean_row.index = ['Average']
sousibyearlytable = pd.concat([sousibyearlytable, mean_row])

# Calculate standard deviation of each column using rows 1-16
std_dev = sousibyearlytable.iloc[:16].std().to_frame().T
std_dev.index = ['Standard Deviation']
sousibyearlytable = pd.concat([sousibyearlytable, std_dev])

# Calculate the standard deviation for each row and create a new column
sousibyearlytable['STD'] = sousibyearlytable.iloc[:, :-1].std(axis=1)

sousibyearlytable

In [ ]:
sousibyearlyredred = sousibyearlyred.copy()
sousibyearlyredred.drop('AVG', axis=1, inplace=True)
sousibyearlyredred = sousibyearlyredred.iloc[:16]

# Initialize a list to store results
results = []

# Iterate through each pair of columns
for col1, col2 in combinations(sousibyearlyredred.columns, 2):
    # Perform T-test
    t_stat, p_value = stats.ttest_ind(sousibyearlyredred[col1], sousibyearlyredred[col2], equal_var=False)
    # Format p_value to 4 decimal places
    p_value = round(p_value, 3)
    # Store the result
    results.append((col1, col2, t_stat, p_value))

# Convert results to a DataFrame
sousib_pval = pd.DataFrame(results, columns=["Column1", "Column2", "T-Statistic", "P-Value"])

# Print the results
print(sousib_pval)

# Print only the statistically relevant results
filtered_nap = sousib_pval[sousib_pval['P-Value'] < 0.05]
print(filtered_nap)

In [ ]:
boxplot = sousibyearlyred.iloc[:16].boxplot(column=['GFED4s', 'GABAM', 'MCD64A1', 'FireCCI51', 'MODIS Active Fires', 'FireCCILT11', 'GFED5'], grid=False, sym='x', vert=False, showmeans=True)
plt.axhline(y=3.5, color='r', linestyle='--')
plt.axhline(y=6.5, color='r', linestyle='--')
plt.xlabel('Burned area (Mha)')
#plt.savefig('FigS9 (SouSib).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
sousibyearlyred['AVG'].iloc[16:] = np.nan

In [ ]:
f = plt.figure(figsize=[20,6])
ax = f.add_subplot(111)
sousibyearlyredgraph = sousibyearlyred.copy()
sousibyearlyredgraph = sousibyearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires']]
sousibyearlyredgraph = sousibyearlyredgraph.iloc[:20]
sousibyearlyred = sousibyearlyred.rename(columns={'AVG': str()})
sousibyearlyred[''].plot(kind='line', ax=ax, color='black', marker='o')
sousibyearlyredgraph.plot(kind='bar', width=0.8, ax=ax)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(sousibyearlyredgraph.columns))
ax.set_xlabel('year')
ax.set_ylabel('Burned area (Mha)')
#plt.savefig('Fig9 (SouSib bar).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
f = plt.figure(figsize=[20,6])
ax = f.add_subplot(111)
sousibyearlyredgraph = sousibyearlyred.copy()
sousibyearlyredgraph['Average'] = sousibyearlyredgraph.mean(axis=1)
sousibyearlyredgraph['Average'].iloc[16:] = np.nan
sousibyearlyredgraph = sousibyearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires', 'Average']]
sousibyearlyredgraph = sousibyearlyredgraph.iloc[:20]
sousibyearlyred = sousibyearlyred.rename(columns={'AVG': str()})
(sousibyearlyredgraph[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires']]
 .plot(kind='line', ax=ax, lw=2, color=col_set))
sousibyearlyredgraph['Average'].plot(kind='line', ax=ax, color='black', marker='o', lw=4, ls='--', ms=8)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=len(sousibyearlyredgraph.columns))
ax.set_xticks(np.arange(len(sousibyearlyredgraph.index)))
ax.set_xticklabels(sousibyearlyredgraph.index)
ax.set_xlabel('year')
ax.set_ylabel('Burned area (Mha)')
#plt.savefig('Fig9 (SouSib line).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
sousibyearlybar = sousibyearlyred.copy()
sousibyearlybar = sousibyearlybar.round(2)

f = plt.figure(figsize=[6,6])
ax = f.add_subplot(111)
#sousibyearlybar = sousibyearlybar[['FireCCI51', 'MCD64A1', 'GABAM', 'GFED4s', 'GFED5', 'FireCCILT11', 'MODIS Active Fires']]
sousibyearlybar = sousibyearlybar[['GFED4s', 'GABAM', 'MCD64A1', 'FireCCI51', 'MODIS Active Fires', 'FireCCILT11', 'GFED5']]
col_sousib = ['#d62728', '#17becf', '#bcbd22', '#8c564b', '#2ca02c', '#9467bd', '#ff7f0e']
#sousibyearlybar.loc['SUM'].plot(kind='bar', ax=ax, color=col_set)
sousibyearlybar.loc['SUM'].plot(kind='bar', ax=ax, color=col_sousib)
ax.set_ylabel('Burned area (Mha)')
ax.bar_label(ax.containers[0])
#plt.savefig('FigS8 (SouSib).tif', dpi=300, bbox_inches='tight', transparent=True)
plt.show()